### Basic SQL Workflow

### Config


In [5]:
import os
from sqlalchemy import create_engine
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv

import sys 
sys.path.append('../')
from helper import get_paths

class Config:
    """SQL agent config"""
    def __init__(
        self,
        openai_model = 'gpt-4-1106-preview',
        temperature  = 0.7
        ):
        path_map  = get_paths()
        env_fpath = path_map['env']
        sql_fpath = path_map['sql']

        load_dotenv(env_fpath)
        openai_api_key = os.getenv("OPENAI_API_KEY")
        
        self.llm = ChatOpenAI(
            model=openai_model,
            temperature=temperature,
            api_key=openai_api_key
            )
        self.db     = SQLDatabase.from_uri(f"sqlite:///{sql_fpath}")
        self.engine = create_engine(f"sqlite:///{sql_fpath}")

config = Config()
verbose = True

### Helpers

In [4]:
import re 

def get_schema_context(config=config, tbls_to_exclude=[]):
    db = config.db
    tables = db.get_usable_table_names()

    schema_lines = []
    for table in tables:
        if table not in tbls_to_exclude:
            table_info = db.get_table_info([table])
            schema_lines.append(f"Table: {table}\n{table_info}\n")

    schema_context = ("DATABASE SCHEMA:\n" + "\n".join(schema_lines))
    return schema_context

def extract_query(response, type='sql'):
    pattern = rf"```{type}\s+([\s\S]*?)\s+```"
    match   = re.search(pattern, response)
    
    if match:
        return match.group(1).strip()
    else:
        print(f"Extracting query of type {type} failed: returning response.strip():\n{response.strip()}")
        return response.strip()

if verbose:
    print(get_schema_context())

DATABASE SCHEMA:
Table: fxrates

CREATE TABLE fxrates (
	date TIMESTAMP, 
	usd_to_eur REAL, 
	usd_to_gbp REAL, 
	usd_to_jpy REAL
)

/*
3 rows from fxrates table:
date	usd_to_eur	usd_to_gbp	usd_to_jpy
2020-01-01 00:00:00	0.9552158220469241	0.7442134597684561	178.67087145000016
2020-01-02 00:00:00	0.9537308256723278	0.7785893129739717	149.17956918593097
2020-01-03 00:00:00	0.9883022675224316	0.8852523415853203	145.90253010300282
*/

Table: ohlc

CREATE TABLE ohlc (
	date TIMESTAMP, 
	open REAL, 
	high REAL, 
	low REAL, 
	close REAL
)

/*
3 rows from ohlc table:
date	open	high	low	close
2020-01-01 00:00:00	97.84704724718776	122.24383711340477	55.44555344062485	92.24425183108863
2020-01-02 00:00:00	92.1025462060139	123.51768361972003	66.15905516369102	95.32120830941909
2020-01-03 00:00:00	106.36100005515581	128.19239423274277	55.33471311840124	86.26686713329055
*/

Table: treasury_yields

CREATE TABLE treasury_yields (
	date TIMESTAMP, 
	yield_5_year REAL, 
	yield_7_year REAL, 
	yield_10_y

### Generating Tables JSON


In [15]:
### one shot 

template = ''' 
    ```json
    {{
    "tables": [
        {{
        "name": "<db_table_name>",
        "columns": [
            {{
            "original_name": "<col_name_in_db>",
            "alias": "<col_alias_in_output>"
            }}
        ]
        }}
    ]}}
    ```
    '''

examples = [{
    "input" : "Calculate the correlation between 7-year treasury yields and stocks' close prices over the last 30 days.",
    "output" : '''        
    ```json
    {{
    "tables": [
        {{
        "name": "treasury_yields",
        "columns": [
            {{
            "original_name": "date",
            "alias": "yield_date"
            }},
            {{
            "original_name": "yield_5_year",
            "alias": "yield_5y_tsy"
            }},
            {{
            "original_name": "yield_10_year",
            "alias": "yield_10y_tsy"
            }}
        ]
        }},
    }}
    ```
    '''
}]

for example in examples:
    print(example['input'])
    print(f'=' * 30)
    print(example['output'])
    print(f'=' * 30)

Calculate the correlation between 7-year treasury yields and stocks' close prices over the last 30 days.
        
    ```json
    {{
    "tables": [
        {{
        "name": "treasury_yields",
        "columns": [
            {{
            "original_name": "date",
            "alias": "yield_date"
            }},
            {{
            "original_name": "yield_5_year",
            "alias": "yield_5y_tsy"
            }},
            {{
            "original_name": "yield_10_year",
            "alias": "yield_10y_tsy"
            }}
        ]
        }},
    }}
    ```
    


In [22]:
import json 
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)

def get_tables_json_str(user_query, config=config, error_msg=None):
    example_prompt = ChatPromptTemplate.from_messages(
        [('human', '{input}'), ('ai', '{output}')]
    )

    few_shot_prompt = FewShotChatMessagePromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
    )
    schema_context = get_schema_context(config)
    system_message  = f'''Given a user query and a SQLite database schema, return ONLY a valid JSON describing the data required to answer the user query. The JSON should be parsable and adhere to proper JSON syntax. Answer ONLY with a JSON in this format: {template}
    Here is the database schema: {schema_context}'''
    final_prompt = ChatPromptTemplate.from_messages(
        [
            ('system', f'{system_message}'),
            few_shot_prompt,
            ('human', '{input}'),
        ]
    )
    chain = final_prompt | config.llm 
    llm_response = chain.invoke({'input' : user_query}).content 
    return llm_response

In [20]:
user_query = f'''For the day where the sum of usd_to_eur, usd_to_gbp, and usd_to_jpy 
    was closest to 150, calculate the weighted average of the EUR equivalent of open, the GBP 
    equivalent of close, and the JPY equivalent of high, with weights being the corresponding 
    treasury yields'''
llm_response = get_tables_json_str(user_query)

In [26]:
tables_json = json.loads(extract_query(llm_response, type='json'))
print(tables_json)

{'tables': [{'name': 'fxrates', 'columns': [{'original_name': 'date', 'alias': 'fx_date'}, {'original_name': 'usd_to_eur', 'alias': 'usd_to_eur'}, {'original_name': 'usd_to_gbp', 'alias': 'usd_to_gbp'}, {'original_name': 'usd_to_jpy', 'alias': 'usd_to_jpy'}]}, {'name': 'ohlc', 'columns': [{'original_name': 'date', 'alias': 'ohlc_date'}, {'original_name': 'open', 'alias': 'open_usd'}, {'original_name': 'high', 'alias': 'high_usd'}, {'original_name': 'close', 'alias': 'close_usd'}]}, {'name': 'treasury_yields', 'columns': [{'original_name': 'date', 'alias': 'ty_date'}, {'original_name': 'yield_5_year', 'alias': 'yield_5_year'}, {'original_name': 'yield_7_year', 'alias': 'yield_7_year'}, {'original_name': 'yield_10_year', 'alias': 'yield_10_year'}]}]}


### Generating Joins JSON

In [29]:
### one shot 

template = ''' 
    ```
    json
    {{
        "join_node": {{
        "join_type": "inner",
        "left": {{
            "join_type": "inner",
            "left": {{
            "table": "{{tbl_A}}"
            }},
            "right": {{
            "table": "{{tbl_B}}"
            }},
            "on_left_key": "{{left_col_to_join_on}}",
            "on_right_key": "{{right_col_to_join_on}}",
            "keep_left": true,
            "keep_right": false
        }},
        "right": {{
            "table": "{{tbl_C}}"
        }},
        "on_left_key": "{{left_join_col}}",
        "on_right_key": "{{right_join_col}}",
        "keep_left": true,
        "keep_right": true
        }}
    }}
    ```
    '''

examples = [{
    "user_query" : "For the day with the lowest ratio of 5y tsy yield to 10y tsy yield among days where USD to GBP was greater than 0.75, calculate the percentage difference between the EUR equivalent of the close price and the JPY equivalent of the open price in the ohlc table.",
    "tables_json" : ''' 
    ```json
        {{
        "tables": [
          {{
            "name": "treasury_yields",
            "columns": [
              {{
                "original_name": "date",
                "alias": "yield_date"
              }},
              {{
                "original_name": "yield_5_year",
                "alias": "yield_5y_tsy"
              }},
              {{
                "original_name": "yield_10_year",
                "alias": "yield_10y_tsy"
              }}
            ]
          }},
          {{
            "name": "fxrates",
            "columns": [
              {{
                "original_name": "date",
                "alias": "fx_date"
              }},
              {{
                "original_name": "usd_to_gbp",
                "alias": "usd_gbp"
              }},
              {{
                "original_name": "usd_to_eur",
                "alias": "usd_eur"
              }},
              {{
                "original_name": "usd_to_jpy",
                "alias": "usd_jpy"
              }}
            ]
          }},
          {{
            "name": "ohlc",
            "columns": [
              {{
                "original_name": "date",
                "alias": "ohlc_date"
              }},
              {{
                "original_name": "open",
                "alias": "stock_open"
              }},
              {{
                "original_name": "close",
                "alias": "stock_close"
              }}
            ]
          }}
        ]
      }}
    ''',
    "output" : '''
    ```json
      {{
      "join_node": {{
        "join_type": "inner",
        "left": {{
          "table": "treasury_yields"
        }},
        "right": {{
          "join_type": "inner",
          "left": {{
            "table": "ohlc"
          }},
          "right": {{
            "table": "fxrates"
          }},
          "on_left_key": "ohlc_date",
          "on_right_key": "fx_date",
          "keep_left": false,
          "keep_right": false
        }},
        "on_left_key": "yield_date",
        "on_right_key": "ohlc_date",
        "keep_left": true,
        "keep_right": false
      }}
    }}
    ```
    '''
    }]

for example in examples:
    print(example['user_query'])
    print(f'=' * 30)
    print(example['tables_json'])
    print(f'=' * 30)
    print(example['output'])

For the day with the lowest ratio of 5y tsy yield to 10y tsy yield among days where USD to GBP was greater than 0.75, calculate the percentage difference between the EUR equivalent of the close price and the JPY equivalent of the open price in the ohlc table.
 
    ```json
        {{
        "tables": [
          {{
            "name": "treasury_yields",
            "columns": [
              {{
                "original_name": "date",
                "alias": "yield_date"
              }},
              {{
                "original_name": "yield_5_year",
                "alias": "yield_5y_tsy"
              }},
              {{
                "original_name": "yield_10_year",
                "alias": "yield_10y_tsy"
              }}
            ]
          }},
          {{
            "name": "fxrates",
            "columns": [
              {{
                "original_name": "date",
                "alias": "fx_date"
              }},
              {{
                "original_name

In [30]:
def get_joins_json_str(user_query, tables_json, config=config, error_msg=None):
    example_prompt = ChatPromptTemplate.from_messages(
        [('human', 'Given user query: {user_query} and tables json: {tables_json}'), ('ai', '{output}')]
    )

    few_shot_prompt = FewShotChatMessagePromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
    )
    schema_context = get_schema_context(config)
    system_message  = f'''Given a user query, a json of table info, return ONLY a valid JSON describing the 
    joins on the tables required to get the data necessary to answer the user query. The JSON should
    be parsable and adhere to proper JSON syntax.
    Answer ONLY with a JSON in this format: {template}
    Instructions:
        - Note the hierarchichal structure, this is to distinguish inner(A, outer(B,C)) != outer(B, inner(A,C))
        - For keep_left, keep_right, only keep a column if absolutely necessary, for example some inner
        joins you won't need both resulting columns
        - don't name any cols the same things
        - Make sure the on_right_key, on_left_key are the OLD names (date, not date_ohlc)
    '''
    final_prompt = ChatPromptTemplate.from_messages(
        [
            ('system', f'{system_message}'),
            few_shot_prompt,
            ('human', 'Given user query: {user_query} and tables json: {tables_json}'),
        ]
    )
    chain = final_prompt | config.llm 
    llm_response = chain.invoke({'user_query' : user_query, "tables_json" : tables_json}).content 
    return llm_response

In [31]:
user_query = f'''For the day where the sum of usd_to_eur, usd_to_gbp, and usd_to_jpy 
    was closest to 150, calculate the weighted average of the EUR equivalent of open, the GBP 
    equivalent of close, and the JPY equivalent of high, with weights being the corresponding 
    treasury yields'''
llm_response = get_joins_json_str(user_query, tables_json)

In [32]:
joins_json = json.loads(extract_query(llm_response, type='json'))
print(joins_json)

{'join_node': {'join_type': 'inner', 'left': {'join_type': 'inner', 'left': {'table': 'fxrates'}, 'right': {'table': 'ohlc'}, 'on_left_key': 'date', 'on_right_key': 'date', 'keep_left': True, 'keep_right': True}, 'right': {'table': 'treasury_yields'}, 'on_left_key': 'fx_date', 'on_right_key': 'date', 'keep_left': False, 'keep_right': True}}


In [34]:

def compile(tables_json, joins_json):
    def get_columns(table_name):
        """Get column aliases for the specified table."""
        for table in tables_json['tables']:
            if table['name'] == table_name:
                return {col['original_name']: col['alias'] for col in table['columns']}
        return {}

    def build_select_clause(table_name, keep_columns):
        """Build the SELECT clause for a given table."""
        columns = get_columns(table_name)
        return [f"{table_name}.{orig_col} AS {alias}" for orig_col, alias in columns.items() if alias in keep_columns]

    def traverse_joins(join_node, keep_columns):
        """Recursively traverse join nodes to build SQL JOINs."""
        if 'table' in join_node:
            table_name = join_node['table']
            keep_left = join_node.get('keep_left', False)
            keep_right = join_node.get('keep_right', False)
            if keep_left or keep_right:
                keep_columns.update(get_columns(table_name).values())
            return table_name, keep_columns

        join_type = join_node['join_type'].upper()
        left_table, left_keep_columns = traverse_joins(join_node['left'], set())
        right_table, right_keep_columns = traverse_joins(join_node['right'], set())

        on_left_key = join_node['on_left_key']
        on_right_key = join_node['on_right_key']
        keep_left = join_node.get('keep_left', False)
        keep_right = join_node.get('keep_right', False)

        if keep_left:
            keep_columns.update(left_keep_columns)
        if keep_right:
            keep_columns.update(right_keep_columns)

        join_clause = (
            f"{left_table} {join_type} JOIN {right_table} "
            f"ON {left_table}.{on_left_key} = {right_table}.{on_right_key}"
        )

        return f"({join_clause})", keep_columns

    # Start with the top-level join node
    final_table, final_keep_columns = traverse_joins(joins_json['join_node'], set())

    # Build the SELECT clause
    select_clauses = []
    for table in tables_json['tables']:
        table_columns = get_columns(table['name'])
        keep_columns = {col for col in final_keep_columns if col in table_columns.values()}
        select_clauses.extend(build_select_clause(table['name'], keep_columns))

    select_clause = ',\n    '.join(select_clauses)

    # Assemble the full SQL query
    query = f"""
    SELECT
        {select_clause}
    FROM
        {final_table}
    """.strip()

    return query

print(compile(tables_json, joins_json))

SELECT
        
    FROM
        ((fxrates INNER JOIN ohlc ON fxrates.date = ohlc.date) INNER JOIN treasury_yields ON (fxrates INNER JOIN ohlc ON fxrates.date = ohlc.date).fx_date = treasury_yields.date)


In [42]:
tables_json

{'tables': [{'name': 'fxrates',
   'columns': [{'original_name': 'date', 'alias': 'fx_date'},
    {'original_name': 'usd_to_eur', 'alias': 'usd_to_eur'},
    {'original_name': 'usd_to_gbp', 'alias': 'usd_to_gbp'},
    {'original_name': 'usd_to_jpy', 'alias': 'usd_to_jpy'}]},
  {'name': 'ohlc',
   'columns': [{'original_name': 'date', 'alias': 'ohlc_date'},
    {'original_name': 'open', 'alias': 'open_usd'},
    {'original_name': 'high', 'alias': 'high_usd'},
    {'original_name': 'close', 'alias': 'close_usd'}]},
  {'name': 'treasury_yields',
   'columns': [{'original_name': 'date', 'alias': 'ty_date'},
    {'original_name': 'yield_5_year', 'alias': 'yield_5_year'},
    {'original_name': 'yield_7_year', 'alias': 'yield_7_year'},
    {'original_name': 'yield_10_year', 'alias': 'yield_10_year'}]}]}

In [43]:
joins_json['join_node']['on_right_key'] = 'ty_date' 

In [55]:
joins_json['join_node']['left']['on_left_key'] = 'fx_date' 
joins_json['join_node']['left']['on_right_key'] = 'ohlc_date' 


In [57]:
joins_json

{'join_node': {'join_type': 'inner',
  'left': {'join_type': 'inner',
   'left': {'table': 'fxrates'},
   'right': {'table': 'ohlc'},
   'on_left_key': 'fx_date',
   'on_right_key': 'ohlc_date',
   'keep_left': True,
   'keep_right': True},
  'right': {'table': 'treasury_yields'},
  'on_left_key': 'fx_date',
  'on_right_key': 'ty_date',
  'keep_left': False,
  'keep_right': True}}

In [60]:
import itertools

def compile(tables_json, joins_json):
    """
    Given:
      tables_json: {
        "tables": [
          {
            "name": <table_name>,
            "columns": [
              { "original_name": <col>, "alias": <col_alias> },
              ...
            ]
          },
          ...
        ]
      }
      joins_json: {
        "join_node": {
          "join_type": <"inner"|"left"|"right"|"full" etc>,
          "left": <TABLE or JOIN_NODE>,
          "right": <TABLE or JOIN_NODE>,
          "on_left_key": <alias-of-left-col>,
          "on_right_key": <alias-of-right-col>,
          "keep_left": <bool>,
          "keep_right": <bool>
        }
      }

    Returns a string containing the SQL query that joins all tables
    and exposes columns/aliases according to 'keep_left'/'keep_right'
    and the tables' columns definitions.
    """

    # 1) Parse the tables_json into a dictionary for quick lookup:
    #    { table_name -> { original_col -> alias_col, ... } }
    table_defs = {}
    for tbl in tables_json["tables"]:
        name = tbl["name"]
        col_map = {}
        for c in tbl["columns"]:
            orig = c["original_name"]
            alias = c["alias"]
            col_map[orig] = alias
        table_defs[name] = col_map

    # A simple generator so sub-selects each get a unique alias
    alias_counter = itertools.count(1)

    def next_subalias():
        return f"sub_{next(alias_counter)}"
    
    # 2) A helper to build a SELECT list given a table's definition
    #    or given a subquery's (which we must have computed).
    #
    #    We return two things:
    #       * "select_expr"    -> string that can go into SELECT ...
    #       * "available_cols" -> dict of { alias -> "subalias.col_name" }
    #         that can be used by an outer query's ON clauses or further SELECT.
    #
    #    We must handle "keep_left"/"keep_right" inside the join
    #    so that if keep is false, we exclude the join key from the outer SELECT list.
    #
    #    However, note we STILL need that join key in the sub-select
    #    (not aliased into final output) so the outer query can do the ON condition.
    #
    #    So for each join node, we’ll produce a sub-select of all the columns that:
    #       a) appear in that side’s final output, or
    #       b) are used as the join key in the next-larger context.
    #
    #    The top-level caller will decide whether a join key is "kept" or not,
    #    and pass that to the recursive call.
    
    def build_from_node(join_node, needed_join_alias=None):
        """
        Recursively build either:
          - A simple "table" select expression,
          - Or a sub-select expression (if there's a nested join_node).

        :param join_node: Either { "table": "some_table" }
                          or a {"join_type", "left", "right", ...} structure.
        :param needed_join_alias: an alias (string) for the column that the
                                  *outer level* needs to join on. (Could be None)

        :return: tuple of:
            (sub_select_sql,        # e.g. "table_name" or "(SELECT ... ) AS sub_X"
             available_cols_dict)   # { alias -> "sub_X.alias" } for use outside
        """

        # If this is just {"table": "some_table"}, no actual join:
        if "table" in join_node:
            table_name = join_node["table"]
            sub_alias = table_name  # or you can rename as you'd like
            col_map = table_defs[table_name]

            # Build the list of original->alias:
            # For a simple table, we just expose all columns by default,
            # but the "outer" node might still discard the join key. So we
            # do the "select * from table AS sub_alias" approach but with explicit columns.
            select_cols = []
            available_cols = {}
            for orig_col, alias_col in col_map.items():
                # sub_alias.orig_col as alias_col
                select_cols.append(f"  {sub_alias}.{orig_col} AS {alias_col}")
                available_cols[alias_col] = f"{sub_alias}.{alias_col}"

            select_expr = ",\n".join(select_cols)
            from_sql = (
                f"(SELECT\n{select_expr}\n"
                f"FROM {table_name} AS {sub_alias}) AS {sub_alias}"
            )
            return from_sql, available_cols

        # Otherwise, it's a join node with structure:
        # {
        #   "join_type": ...,
        #   "left": { table or sub-join },
        #   "right": { table or sub-join },
        #   "on_left_key": ...,
        #   "on_right_key": ...,
        #   "keep_left": bool,
        #   "keep_right": bool
        # }
        join_type = join_node["join_type"].upper() + " JOIN"
        left_node = join_node["left"]
        right_node = join_node["right"]
        on_left_key_alias = join_node["on_left_key"]   # e.g. "ohlc_date"
        on_right_key_alias = join_node["on_right_key"] # e.g. "fx_date"
        keep_left = join_node["keep_left"]
        keep_right = join_node["keep_right"]

        # We'll recursively build sub-select for left side and right side.
        # But each side might need to "expose" its join key column so that
        # the *outer* ON condition can see it.  We'll rename these columns
        # in the sub-select as well (like `_join_key_left`, `_join_key_right`) to avoid collisions.
        #
        # We do that by telling the child node: "please give me <col_alias> in your output
        # because I'm going to use it in the parent's ON condition".
        #
        # Then, after we get the child's sub-select, if keep_left == False (for left side),
        # we remove that from the parent's *final* columns. But we needed it to be present
        # in that subquery's SELECT so we can do the join.

        left_sql, left_cols = build_from_node(left_node, needed_join_alias=on_left_key_alias)
        right_sql, right_cols = build_from_node(right_node, needed_join_alias=on_right_key_alias)

        # The child subqueries each have "available_cols" as { alias -> subX.alias }.
        # Now we can form an ON condition:
        left_join_col = left_cols[on_left_key_alias]
        right_join_col = right_cols[on_right_key_alias]
        on_condition = f"{left_join_col} = {right_join_col}"

        # Now we want to build a sub-select that includes columns from both sides:
        # We'll combine all columns from left_cols and right_cols,
        # possibly removing the join keys if keep_left/right are false.
        combined_select_lines = []
        combined_cols = {}

        # Include all columns from left side:
        for alias_col, fully_qualified in left_cols.items():
            # If this alias_col is the left join key and keep_left is False, skip including in final:
            if alias_col == on_left_key_alias and not keep_left:
                continue
            # else keep it
            combined_select_lines.append(f"  {fully_qualified} AS {alias_col}")
            combined_cols[alias_col] = alias_col  # will refer back to sub-alias from outer

        # Include all columns from right side:
        for alias_col, fully_qualified in right_cols.items():
            # If this alias_col is the right join key and keep_right is False, skip:
            if alias_col == on_right_key_alias and not keep_right:
                continue
            # else keep it
            combined_select_lines.append(f"  {fully_qualified} AS {alias_col}")
            combined_cols[alias_col] = alias_col

        # We'll create a new sub-select from left_sql JOIN right_sql ON condition
        sub_alias = next_subalias()

        sub_select_sql = (
            f"(\n"
            f"  SELECT\n"
            f"{',\n'.join(combined_select_lines)}\n"
            f"  FROM {left_sql}\n"
            f"  {join_type} {right_sql}\n"
            f"    ON {on_condition}\n"
            f") AS {sub_alias}"
        )

        # The "available_cols" we return need to be in the form
        # { alias -> "sub_alias.alias" } so that the outer query can reference them
        final_available_cols = {
            alias_col: f"{sub_alias}.{alias_col}" for alias_col in combined_cols
        }

        return sub_select_sql, final_available_cols

    # 3) The top-level of joins_json has the structure: {"join_node": {...}}
    root_join_node = joins_json["join_node"]
    top_sql, top_cols = build_from_node(root_join_node)

    # The top-level sub-select is everything.  Now we wrap it in a final SELECT
    # so that we produce a "flat" list of columns.  The question is:
    # do we simply SELECT * from (top_sql)? Or do we want to pick the columns
    # in the order in which they appear in the table_defs? Typically, your
    # requirement was “pull each of the cols as aliases” in the final, which
    # is effectively all aliases in `top_cols`.
    #
    # Here we’ll just pull them in alphabetical order for consistency, or you
    # could preserve the table order. We skip “hidden” join aliases if they
    # got pruned out, but we only have those columns that remain in top_cols anyway.

    # In many real-world cases, you can just do: SELECT sub_1.* FROM ( ... ) sub_1
    # but we’ll be explicit:
    final_alias = "final"
    output_select_lines = []
    # Sort columns to have stable ordering (not required, but can be nice):
    for alias_col in sorted(top_cols.keys()):
        output_select_lines.append(f"  {top_cols[alias_col]} AS {alias_col}")

    final_query = (
        f"SELECT\n"
        f"{',\n'.join(output_select_lines)}\n"
        f"FROM {top_sql}\n"
    )

    return final_query

sql_query = compile(tables_json, joins_json)

import pandas as pd 
df = pd.read_sql(sql_query, config.engine)
df.head()

,close_usd,high_usd,ohlc_date,open_usd,ty_date,usd_to_eur,usd_to_gbp,usd_to_jpy,yield_10_year,yield_5_year,yield_7_year
0,92.244252,122.243837,2020-01-01 00:00:00,97.847047,2020-01-01 00:00:00,0.955216,0.744213,178.670871,2.420872,1.545694,3.533141
1,95.321208,123.517684,2020-01-02 00:00:00,92.102546,2020-01-02 00:00:00,0.953731,0.778589,149.179569,2.566896,2.245038,2.461981
2,86.266867,128.192394,2020-01-03 00:00:00,106.361000,2020-01-03 00:00:00,0.988302,0.885252,145.902530,3.327936,1.397994,4.226303
3,109.121746,125.600137,2020-01-06 00:00:00,80.813697,2020-01-06 00:00:00,0.898221,0.776084,127.671146,4.022134,2.529819,1.773818
4,116.741645,125.857878,2020-01-07 00:00:00,110.707861,2020-01-07 00:00:00,0.890543,0.760646,134.947739,3.628586,1.763754,3.660253


In [61]:
def run_sql_agent(user_query, config):
    table_json_str = get_tables_json_str(user_query, config)
    tables_json = json.loads(extract_query(llm_response, type='json'))
    joins_json_str = get_joins_json_str(user_query, tables_json, config)
    joins_json = json.loads(extract_query(llm_response, type='json'))
    df = compile(tables_json, joins_json)
    return df